In [ ]:
from transformers import pipeline
import json

# Load REBEL model from Hugging Face (text2text-generation)
print("🔄 Loading model...")
rebel = pipeline("text2text-generation", model="Babelscape/rebel-large", tokenizer="Babelscape/rebel-large")



/home/radzz/dl-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
/home/radzz/dl-env/lib/python3.12/site-packages/transformers

OSError: explodinggradients/roberta-base-relation-extraction is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
# 📝 Input paragraph
print("📝 Enter a paragraph (press Enter twice to finish):")
lines = []
while True:
    line = input()
    if not line.strip():
        break
    lines.append(line)

text = " ".join(lines).strip()
if not text:
    print("⚠️ No input provided.")
    exit(1)

# 🔍 Step 1: Named entity extraction
ner_results = ner_pipeline(text)
entities = [ent['word'].replace("##", "").strip() for ent in ner_results if ent['score'] > 0.85]

# 🧠 Clean duplicates and group similar names
entities = list(set(entities))
entities = sorted(entities)

# ⛓️ Step 2: Create (subject, object) pairs
pairs = list(itertools.permutations(entities, 2))  # All possible directed pairs

edges = []

# 🔎 Step 3: Run relation extraction on each pair
print(f"🔗 Checking {len(pairs)} possible entity pairs...")
for subj, obj in pairs:
    prompt = f"{subj} [SEP] {obj} [SEP] {text}"
    inputs = rel_tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = rel_model(**inputs)
    pred_id = torch.argmax(outputs.logits, dim=1).item()
    relation = rel_model.config.id2label[pred_id]

    if relation != "no_relation":
        edges.append({
            "source": subj,
            "target": obj,
            "relation": relation.replace("_", " ").capitalize()
        })

# 💾 Step 4: Save to edges.json
with open("edges.json", "w") as f:
    json.dump(edges, f, indent=2)

# ✅ Final output
print(f"\n✅ Extracted {len(edges)} valid relation(s):")
for edge in edges:
    print(f"{edge['source']} —[{edge['relation']}]→ {edge['target']}")